## Задание 5.1

Набор данных тут: https://github.com/sismetanin/rureviews, также есть в папке [Data](https://drive.google.com/drive/folders/1YAMe7MiTxA-RSSd8Ex2p-L0Dspe6Gs4L). Те, кто предпочитает работать с английским языком, могут использовать набор данных `sms_spam`.

Применим полученные навыки и решим задачу анализа тональности отзывов. 

Нужно повторить весь пайплайн от сырых текстов до получения обученной модели.

Обязательные шаги предобработки:
1. токенизация
2. приведение к нижнему регистру
3. удаление стоп-слов
4. лемматизация
5. векторизация (с настройкой гиперпараметров)
6. построение модели
7. оценка качества модели

Обязательно использование векторайзеров:
1. мешок n-грамм (диапазон для n подбирайте самостоятельно, запрещено использовать только униграммы).
2. tf-idf ((диапазон для n подбирайте самостоятельно, также нужно подбирать параметры max_df, min_df, max_features)
3. символьные n-граммы (диапазон для n подбирайте самостоятельно)

В качестве классификатора нужно использовать наивный байесовский классификатор. 

Для сравнения векторайзеров между собой используйте precision, recall, f1-score и accuracy. Для этого сформируйте датафрейм, в котором в строках будут разные векторайзеры, а в столбцах разные метрики качества, а в  ячейках будут значения этих метрик для соответсвующих векторайзеров.

In [ ]:
%%capture
!wget https://www.dropbox.com/s/2yms4xgtuvex3gx/women-clothing-accessories.csv
!pip install pymorphy2
import pandas as pd
import numpy as np
import nltk
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import *
import re
data = pd.read_csv("women-clothing-accessories.csv", sep='\t')
nltk.download('stopwords')
nltk.download('punkt')

In [ ]:
analyzer = MorphAnalyzer()

def bayes(vectorizer, vectorizer_x_train):
  clf = MultinomialNB()
  clf.fit(vectorizer_x_train, y_train)  
  vectorizer_x_test = vectorizer.transform(X_test)
  pred = clf.predict(vectorizer_x_test)
  return classification_report(y_test, pred, output_dict=True), vectorizer, pred

def processing(data):
  sentences = []
  for sentence in data:
    sentence = sentence.lower()
    for ch in string.punctuation:
      sentence = sentence.replace(ch,"")
    sentence = re.sub(r'\d', '', sentence)
    words = word_tokenize(sentence)
    for i in range(len(words)):
      words[i] = analyzer.parse(words[i])[0].normal_form
    sentences.append(" ".join(words))
  return sentences

data.review = processing(data.review)
X_train, X_test, y_train, y_test = train_test_split(data.review, data.sentiment, train_size = 0.7)

In [ ]:
vectorized_info = []

for i in range(1, 7):#(1,6)
  for j in range(i + 1, 10):#(i+1,6)
    vectorizer = CountVectorizer(ngram_range=(i,j), analyzer="word", stop_words=stopwords.words("russian"))
    vectorizer_x_train = vectorizer.fit_transform(X_train)
    a = bayes(vectorizer, vectorizer_x_train)
    vectorized_info.append([a, str(vectorizer)])


for i in range(5, 12):#(3,6)
  for j in range(i + 1, 6):#(i + 1,6)
    vectorizer = CountVectorizer(ngram_range=(i,j), analyzer="char", stop_words=stopwords.words("russian"))
    vectorizer_x_train = vectorizer.fit_transform(X_train)
    a = bayes(vectorizer, vectorizer_x_train)
    vectorized_info.append([a, str(vectorizer)])

In [ ]:
#В бустинге - градиент в пространстве модолей

In [ ]:
for i in range(1, 4):
  for j in range(i + 1, 4):
    for max_df in [0.2, 0.4, 0.6]:#0.7/1.0/0.1
      for min_df in [0.01, 0.02]:
        for max_features in [2000, 4000, 8000]:
          vectorizer = TfidfVectorizer(ngram_range=(i,j), max_df=max_df, min_df=min_df, max_features=max_features)
          vectorizer_x_train = vectorizer.fit_transform(X_train)
          a = bayes(vectorizer, vectorizer_x_train)
          vectorized_info.append([a, str(vectorizer)])

In [ ]:
results = []
pattern = re.compile(r"[a-zA-Z]+Vectorizer|'word'|'char'|\([0-9,]+,[ 0-9]+\)|min_df=[0-9.]|max_df=[0-9.]+|features=[0-9]+")
for vector_info in vectorized_info:
  metrics = vector_info[0][0]['weighted avg']
  parameters = pattern.findall(vector_info[1])
  results.append({"Vectorizer": parameters[0], "Parameters": parameters[1:], "Precision": metrics["precision"], "Recall": metrics["recall"], "F1-Score": metrics['f1-score'], "Accuracy": vector_info[0][0]['accuracy']})
data_result = pd.DataFrame(results)
data_result.sort_values(by=["Accuracy"], ascending=False).head(20)

,Vectorizer,Parameters,Precision,Recall,F1-Score,Accuracy
0,CountVectorizer,"[(1, 2)]",0.711296,0.708455,0.708764,0.708455
12,CountVectorizer,"['char', (4, 5)]",0.717458,0.705863,0.707919,0.705863
1,CountVectorizer,"[(1, 3)]",0.707111,0.705826,0.705839,0.705826
2,CountVectorizer,"[(1, 4)]",0.706199,0.705270,0.705242,0.705270
3,CountVectorizer,"[(1, 5)]",0.705937,0.705159,0.705111,0.705159
11,CountVectorizer,"['char', (3, 5)]",0.715690,0.703122,0.705388,0.703122
10,CountVectorizer,"['char', (3, 4)]",0.707426,0.694604,0.696891,0.694604
25,TfidfVectorizer,"[max_df=0.6, features=2000, min_df=0, (1, 2)]",0.689568,0.675568,0.678225,0.675568
27,TfidfVectorizer,"[max_df=0.6, features=8000, min_df=0, (1, 2)]",0.689568,0.675568,0.678225,0.675568
26,TfidfVectorizer,"[max_df=0.6, features=4000, min_df=0, (1, 2)]",0.689568,0.675568,0.678225,0.675568


## Задание 5.2 Регулярные выражения

Регулярные выражения - способ поиска и анализа строк. Например, можно понять, какие даты в наборе строк представлены в формате DD/MM/YYYY, а какие - в других форматах. 

Или бывает, например, что перед работой с текстом, надо почистить его от своеобразного мусора: упоминаний пользователей, url и так далее.

Навык полезный, давайте в нём тоже потренируемся.

Для работы с регулярными выражениями есть библиотека **re**

In [ ]:
import re

В регулярных выражениях, кроме привычных символов-букв, есть специальные символы:
* **?а** - ноль или один символ **а**
* **+а** - один или более символов **а**
* **\*а** - ноль или более символов **а** (не путать с +)
* **.** - любое количество любого символа

Пример:
Выражению \*a?b. соответствуют последовательности a, ab, abc, aa, aac НО НЕ abb!

Рассмотрим подробно несколько наиболее полезных функций:

### findall
возвращает список всех найденных непересекающихся совпадений.

Регулярное выражение **ab+c.**: 
* **a** - просто символ **a**
* **b+** - один или более символов **b**
* **c** - просто символ **c**
* **.** - любой символ


In [ ]:
result = re.findall('ab+c.', 'atdsytrcjytabcxabcdavu\tyjvgkabcdartuyfgabbbcdavbhi8h.ojp;') 
print(result)

['abcx', 'abcd', 'abcd', 'abbbcd']


Вопрос на внимательность: почему нет abcx?

**Задание**: вернуть список первых двух букв каждого слова в строке, состоящей из нескольких слов.

In [ ]:
result = re.findall(r'[^-]\b\w\w', 'there is a lot of strangers at night')
result

[' is', ' lo', ' of', ' st', ' at', ' ni']

### split
разделяет строку по заданному шаблону


In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie') 
print(result)

['itsy', ' bitsy', ' teenie', ' weenie']


можно указать максимальное количество разбиений

In [ ]:
result = re.split(',', 'itsy, bitsy, teenie, weenie', maxsplit=2) 
print(result)

['itsy', ' bitsy', ' teenie, weenie']


**Задание**: разбейте строку, состоящую из нескольких предложений, по точкам, но не более чем на 3 предложения.

In [ ]:
string = 'Two roads diverged in a yellow wood, and sorry I could not travel both. And be one traveler, long I stood. And looked down one as far as I could to where it bent in the undergrowth.'
result = re.split('\.', string , maxsplit=2) 
result

['Two roads diverged in a yellow wood, and sorry I could not travel both',
 ' And be one traveler, long I stood',
 ' And looked down one as far as I could to where it bent in the undergrowth.']

### sub
ищет шаблон в строке и заменяет все совпадения на указанную подстроку

параметры: (pattern, repl, string)

In [ ]:
result = re.sub('a', 'b', 'abcabc')
print (result)

bbcbbc


**Задание**: напишите регулярное выражение, которое позволит заменить все цифры в строке на "DIG".

In [ ]:
string = 'I am the 2 in the list, a bird has 2 willows. 123'
result = re.sub('[1-9+]', 'DIG', string)
result

'I am the DIG in the list, a bird has DIG willows. DIGDIGDIG'

**Задание**: напишите  регулярное выражение, которое позволит убрать url из строки.

In [ ]:
string = 'Is it my vk profile link: https://vk.com/feed'
result = re.sub('https://[^\s]+', '', string)
result

'Is it my vk profile link: '

### compile
компилирует регулярное выражение в отдельный объект

In [ ]:
# Пример: построение списка всех слов строки:
prog = re.compile('[А-Яа-яё\-]+')
prog.findall("Слова? Да, больше, ещё больше слов! Что-то ещё.")

['Слова', 'Да', 'больше', 'ещё', 'больше', 'слов', 'Что-то', 'ещё']

**Задание**: для выбранной строки постройте список слов, которые длиннее трех символов.

In [ ]:
prog = re.compile('[А-Яа-яё\-]{4,}')
prog.findall("Кружка стоит на столе, шар, лом, мел")

['Кружка', 'стоит', 'столе']

**Задание**: вернуть список доменов (@gmail.com) из списка адресов электронной почты:

```
abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz
```

In [ ]:
prog = re.compile('.+@gmail.com')
prog.findall('abc.test@gmail.com, xyz@test.in, test.first@analyticsvidhya.com, first.test@rest.biz')

['abc.test@gmail.com']